Установим все необходимые библиотеки

In [1]:
!pip install --upgrade transformers[torch] scikit-learn datasets evaluate sentencepiece

  Obtaining dependency information for transformers[torch] from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 4.7 MB/s eta 0:00:00
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/d0/0b/26ad95cf0b747be967b15fb71a06f5ac67aba0fd2f9cd174de6edefc4674/scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-h

Рассмотрим простой пример того, как работает код, скопированный для случайной модели.

In [1]:
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertPreTrainedModel, DistilBertConfig
from transformers import TrainingArguments, Trainer, PretrainedConfig
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
from transformers import pipeline
from transformers.modeling_outputs import ModelOutput
import evaluate
from datasets import Dataset
from sklearn import datasets
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from torch.nn import CrossEntropyLoss
import numpy as np
from typing import Dict, List, Optional, Set, Tuple, Union
from dataclasses import dataclass

# BERT Finetuning

In [2]:
# загружаем токенайзер, обученный с этой конкретной моделью
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased') 

# загружаем саму модель
model = DistilBertModel.from_pretrained("distilbert-base-uncased") 

# текст, который мы будем использовать как тестовый
text = "This is the text you'd like to test." 

# pt - подготовить выход как тензоры в pytorch, без этого результат был бы обычным списком
encoded_input = tokenizer(text, return_tensors='pt') 

# подаем результат на вход модели, получаем выход
output = model(**encoded_input) 

#  закодированный текст, разбитый на токены, токены спрятаны за id
print(encoded_input['input_ids'].shape) 

# attention_mask позволяет понять где реальные токены, 
# а где pad или другой мусор (0 - игнорим, 1 - учитываем), 
# помогает когда выравниваем несколько текстов под один размер в батче
print(encoded_input['attention_mask'].shape) 

#  закодированные слова, которые используются для классификации, 
# shape определяется как (размер_батча, длина_текстов_в_батче, размерность модели(768))
print(output.last_hidden_state.shape) 
encoded_input, output

torch.Size([1, 13])
torch.Size([1, 13])
torch.Size([1, 13, 768])


({'input_ids': tensor([[ 101, 2023, 2003, 1996, 3793, 2017, 1005, 1040, 2066, 2000, 3231, 1012,
           102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 BaseModelOutput(last_hidden_state=tensor([[[-0.0676, -0.1705,  0.0877,  ..., -0.1348,  0.2474,  0.3418],
          [-0.2672, -0.2979, -0.0951,  ..., -0.2130,  0.4530,  0.2286],
          [-0.2182, -0.2744,  0.2702,  ..., -0.0527,  0.1585,  0.7582],
          ...,
          [ 0.3567,  0.2075, -0.0271,  ..., -0.3326,  0.2028, -0.5782],
          [ 0.3318, -0.2018, -0.3674,  ...,  0.2087, -0.2426, -0.3973],
          [-0.1503, -0.2454,  0.3643,  ..., -0.0560,  0.0665,  0.1469]]],
        grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None))

Но всё, что нам нужно, уже реализовано до нас.

In [3]:
# вот так трансформеры умеют в автомодель/автотокенизер
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased') 

# по умолчанию берет Fast Tokenizer, 
# которые реализованы на Rust и работают быстрее (в отличие от дефолтных на торче)
print(type(tokenizer)) 

<class 'transformers.models.distilbert.tokenization_distilbert_fast.DistilBertTokenizerFast'>


CLS токен используется для классификации, SEP для разделения двух частей входных данных (к примеру, вопрос-ответ):

In [4]:
output = tokenizer("This is the text you'd like to test", "And you can add any sentence youlike!", "The third sentence is ignored")
tokenizer.decode(output.input_ids)

"[CLS] this is the text you'd like to test [SEP] and you can add any sentence youlike! [SEP]"

In [5]:
ng20 = datasets.fetch_20newsgroups() # стандартная коллекция из новостных документов, разбитых на 20 различных тем

### Классическое обучение

In [6]:
input_ids = []
attention_masks = []
labels = []


for i in range(len(ng20.target)):
    encoded = tokenizer.encode_plus(
                        ng20.data[i],                   # предложение которое кодируем
                        padding='max_length',           # подгоняем/обрезаем все тексты до одной длины
                        max_length = 100,               # просто потому что дефолтный 512 слишком долго
                        pad_to_max_length = True,       # добиваем короткие документы до нужной длины
                        truncation=True,                # а слишком короткие просто обрезаем
                        return_attention_mask = True,   # ибо нам нужно знать где реальные токены
                        return_tensors = 'pt',          # вернуть не списки а тензоры
                   )
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])
    labels.append(ng20.target[i])

# собираем из этого большие тензоры
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)


ds = TensorDataset(input_ids, attention_masks, labels)

train_dl = DataLoader(ds, batch_size = 16)

# тут также заметим что есть волшебный класс, который:
# 1. Реализует классификатор с использованием модели, которую мы выбрали
# 2. Конкретный классификатор тоже может подбираться автоматически
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = len(ng20.target_names), output_hidden_states = False)
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-5)

model.train()
for step, batch in tqdm(enumerate(train_dl)):
    b_input_ids = batch[0]
    b_input_mask = batch[1]
    b_labels = batch[2]
    optimizer.zero_grad()
    output = model(input_ids=b_input_ids,
                    attention_mask=b_input_mask,
                    labels=b_labels)
    loss = output.loss # лосс считается самим классификатором, нам можно об этом не волноваться
    loss.backward()
    optimizer.step()
    if step > 10: # просто чтобы остановить это безобразие
        break


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
11it [00:28,  2.63s/it]


### HuggingFace🤗 обучение

In [6]:
ng20 = datasets.fetch_20newsgroups()

# поскольку нейронка понимает только таргеты как целые-числа, говорим, что такие словари - это стандартное решение, оно пойдет на вход классификатору
id2label = {i: label for i, label in enumerate(ng20.target_names)}
label2id = {label: i for i, label in enumerate(ng20.target_names)}

train_dataset = Dataset.from_dict({
    "label" : ng20.target,
    "text" : ng20.data
})

train_tokenized_dataset = train_dataset.map(lambda item: tokenizer(item["text"], truncation=True), batched=True)

Map:   0%|          | 0/11314 [00:00<?, ? examples/s]

Готовим нужную модель, указывая необходимые параметры:

In [8]:
# теперь модель можно инициировать так:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(id2label.keys()), id2label=id2label, label2id=label2id
).to('cuda')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Теперь можно переходить к самому обучению

In [9]:
training_args = TrainingArguments(
    output_dir='/tmp/where_to_save_the_model', # без этого никак - это где сохраняется обученная модель, но можно заигнорить
    learning_rate=1e-5,
    num_train_epochs=1,
    per_device_train_batch_size=16,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # передаем тот самый коллатор, который за нас сможет выровнять все тексты
)

trainer.train()

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.058000


TrainOutput(global_step=708, training_loss=1.8618160183146848, metrics={'train_runtime': 150.2478, 'train_samples_per_second': 75.302, 'train_steps_per_second': 4.712, 'total_flos': 1498326948328320.0, 'train_loss': 1.8618160183146848, 'epoch': 1.0})

Смотрим, чему научилась модель 

In [10]:
test_sentence = "This is just a sentence about politics, president, government and other things."
model.eval() # обязательно переключаем, не забываем!
test_output = model(**tokenizer(test_sentence, return_tensors='pt').to('cuda'))
predicted_class_id = test_output.logits.argmax().item() # по логитам смотрим на наиболее вероятный класс
print(test_output.logits)
print(test_output.logits[0][predicted_class_id])
model.config.id2label[predicted_class_id]

tensor([[ 0.6451, -0.2123, -0.5088, -0.2337, -0.5312, -0.3762, -0.1918, -0.3357,
         -0.1712, -0.1112, -0.1079, -0.0035, -0.1108,  0.1713, -0.0025,  0.2081,
          0.2906,  0.6277,  0.5350, -0.0246]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor(0.6451, device='cuda:0', grad_fn=<SelectBackward0>)


'alt.atheism'

Все уже реализовано за нас

In [11]:
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, device='cuda')
classifier(test_sentence)

[{'label': 'alt.atheism', 'score': 0.09182293713092804}]

## Валидация

In [7]:
ng20_test = datasets.fetch_20newsgroups(subset='test')

test_dataset = Dataset.from_dict({
    "label" : ng20.target,
    "text" : ng20.data
    })

test_tokenized_dataset = test_dataset.map(lambda item: tokenizer(item["text"], truncation=True), batched=True)

Map:   0%|          | 0/11314 [00:00<?, ? examples/s]

Теперь будем обучать нашу модель более правильным способом.

https://huggingface.co/docs/transformers/v4.35.2/en/main_classes/trainer#transformers.TrainingArguments

In [8]:
accuracy = evaluate.load("accuracy") # готовый метод для оценки качества
f1 = evaluate.load("f1")             # готовый метод для оценки f1

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(id2label.keys()), id2label=id2label, label2id=label2id
)

# эта функция будет периодически использоваться во время обучения 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy.compute(predictions=predictions, references=labels)['accuracy'],
        'f1': f1.compute(predictions=predictions, references=labels, average='macro')['f1']
    }


training_args = TrainingArguments(
    output_dir='/tmp/where_to_save_the_model',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128, # поскольку на шаге eval не нужно считать loss, граф вычислений не строится и обычно можно использовать батч побольше
    num_train_epochs=1,
    weight_decay=0.01, # https://medium.com/unpackai/stay-away-from-overfitting-l2-norm-regularization-weight-decay-and-l1-norm-regularization-795bbc5cf958
    warmup_steps = 100, # число шагов (батчей) за которые lr увеличивается от 0 до заданного нами
    lr_scheduler_type = 'linear', # эта штука "постепенно" скручивает learning rate по ходу эпох. linear - дефолтное значение
    evaluation_strategy="steps", # выбираем стратегию оценки качества модели - каждые n батчей (как альтернатива - по эпохам)
    eval_steps=200, # раз в сколько батчей оцениваем
    seed=42, # просто немного для стабильности
    fp16=True, #  очень вполезная штука, заметно скоряет обучение, хотя работает только на GPU
    logging_strategy='steps',
    logging_steps=500, # как часто считаем лос на трейне
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset, # тот самый валидационный датасет
    tokenizer=tokenizer, # токенайзер не обучается, поэтому используем тот же самый
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # сюда суем тот самый коллатор который правильно готовит нам батчи
    compute_metrics=compute_metrics, # та самая функция что мы выше создали
)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1
200,No log,2.115275,0.623829,0.571167
400,No log,1.463692,0.713806,0.674770
600,2.110900,1.238376,0.764363,0.734995


TrainOutput(global_step=708, training_loss=1.884704632947674, metrics={'train_runtime': 109.2218, 'train_samples_per_second': 103.587, 'train_steps_per_second': 6.482, 'total_flos': 1498326948328320.0, 'train_loss': 1.884704632947674, 'epoch': 1.0})

### Hand-made BERT for sequence classification using HF

In [9]:
# Создаем класс для выхода нашей модели: возвращаем сразу два логита
@dataclass
class SequenceDoubleClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits_1: torch.FloatTensor = None
    logits_2: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

# Также создаем новый конфиг для этой модели
class DoubleClassConfig(DistilBertConfig):
    def __init__(self, num_labels_1=None, num_labels_2=None, **kwargs): #None тут важно!
        super().__init__(**kwargs)
        self.num_labels_1 = num_labels_1
        self.num_labels_2 = num_labels_2

# определяем эту самую модель
class DistilBertForSequenceDoubleClassification(DistilBertPreTrainedModel):
    def __init__(self, config: DoubleClassConfig):
        super().__init__(config)
        self.num_labels_1 = config.num_labels_1
        self.num_labels_2 = config.num_labels_2
        self.config = config

        self.distilbert = DistilBertModel(config)

        self.pre_classifier1 = nn.Linear(config.dim, config.dim)
        self.classifier1 = nn.Linear(config.dim, config.num_labels_1)

        self.pre_classifier2 = nn.Linear(config.dim, config.dim)
        self.classifier2 = nn.Linear(config.dim, config.num_labels_2)

        self.dropout = nn.Dropout(config.seq_classif_dropout)

        # Initialize weights and apply final processing
        self.post_init()

    def get_position_embeddings(self) -> nn.Embedding:
        return self.distilbert.get_position_embeddings()

    def resize_position_embeddings(self, new_num_position_embeddings: int):
        self.distilbert.resize_position_embeddings(new_num_position_embeddings)

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        label_1: Optional[torch.LongTensor] = None,
        label_2: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[SequenceDoubleClassifierOutput, Tuple[torch.Tensor, ...]]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        distilbert_output = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_state = distilbert_output[0]
        pooled_output = hidden_state[:, 0]

        pooled_output_1 = self.pre_classifier1(pooled_output) #  эту логику берем прямо из классификатора
        pooled_output_1 = nn.ReLU()(pooled_output_1)
        pooled_output_1 = self.dropout(pooled_output_1)
        logits_1 = self.classifier1(pooled_output_1)

        pooled_output_2 = self.pre_classifier2(pooled_output)
        pooled_output_2 = nn.ReLU()(pooled_output_2)
        pooled_output_2 = self.dropout(pooled_output_2)
        logits_2 = self.classifier2(pooled_output_2)

        loss = None
        if label_1 is not None and label_2 is not None:

            loss_fct = CrossEntropyLoss()
            loss_1 = loss_fct(logits_1.view(-1, self.num_labels_1), label_1.view(-1))
            loss_2 = loss_fct(logits_2.view(-1, self.num_labels_2), label_2.view(-1))

            loss = loss_1 + loss_2

        if not return_dict:
            output = (logits_1,logits_2) + distilbert_output[1:]
            return ((loss,) + output) if loss is not None else output

        return SequenceDoubleClassifierOutput(
            loss=loss,
            logits_1=logits_1,
            logits_2=logits_2,
            hidden_states=distilbert_output.hidden_states,
            attentions=distilbert_output.attentions,
        )



Протестируем как это работает. Создадим игрушечные датасеты:

In [10]:
# Нужно пересобрать наш датасет, чтобы каждому документу соответствовало 2 лейбла

train_2_class_dataset = Dataset.from_dict({
    "label_1" : ng20.target,
    "label_2" : ng20.target % 5,  # просто для примера
    "text" : ng20.data
    })

ng20_test = datasets.fetch_20newsgroups(subset='test')

test_2_class_dataset = Dataset.from_dict({
    "label_1" : ng20_test.target,
    "label_2" : ng20_test.target % 5,  # просто для примера
    "text" : ng20_test.data
    })

train_2_class_tokenized_dataset = train_2_class_dataset.map(lambda item: tokenizer(item["text"], truncation=True), batched=True)
test_2_class_tokenized_dataset = test_2_class_dataset.map(lambda item: tokenizer(item["text"], truncation=True), batched=True)

Map:   0%|          | 0/11314 [00:00<?, ? examples/s]

Map:   0%|          | 0/7532 [00:00<?, ? examples/s]

Инициализируем нашу модель

In [11]:
config = DoubleClassConfig.from_pretrained("distilbert-base-uncased", num_labels_1=len(ng20.target_names), num_labels_2=len(ng20.target_names))
model = DistilBertForSequenceDoubleClassification.from_pretrained(
        "distilbert-base-uncased",
        config=config,
    )

Some weights of DistilBertForSequenceDoubleClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier1.weight', 'classifier1.bias', 'classifier2.weight', 'pre_classifier2.bias', 'pre_classifier1.bias', 'pre_classifier2.weight', 'classifier2.bias', 'classifier1.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

# переписываем метрику, поскольку теперь нам нужно предсказывать сразу 2 класса
def compute_metrics(eval_pred):
    predictions_1, predictions_2 = eval_pred[0]
    labels_1, labels_2 = eval_pred[1]

    predictions_1 = np.argmax(predictions_1, axis=1)
    predictions_2 = np.argmax(predictions_2, axis=1)

    return {
        'accuracy_1': accuracy.compute(predictions=predictions_1, references=labels_1)['accuracy'],
        'f1_1': f1.compute(predictions=predictions_1, references=labels_1, average='macro')['f1'],
        'accuracy_2': accuracy.compute(predictions=predictions_2, references=labels_2)['accuracy'],
        'f1_2': f1.compute(predictions=predictions_2, references=labels_2, average='macro')['f1']
    }

training_args = TrainingArguments(
    output_dir='/tmp/where_to_save_the_model',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=128,
    num_train_epochs=2,
    weight_decay=0.01,
    warmup_steps = 100,
    logging_strategy='steps',
    logging_steps=200,
    lr_scheduler_type = 'linear',
    evaluation_strategy="steps",
    eval_steps=200,
    seed=42,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_2_class_tokenized_dataset,
    eval_dataset=test_2_class_tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics = compute_metrics
)

trainer.train()

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss,Accuracy 1,F1 1,Accuracy 2,F1 2
200,1.449700,1.457490,0.770048,0.740061,0.806293,0.804366
400,1.137900,1.286927,0.791689,0.770522,0.824748,0.823709
600,0.915400,1.213132,0.800982,0.789891,0.831386,0.831229
800,0.770900,1.130790,0.817313,0.809784,0.839618,0.839971
1000,0.639500,1.115498,0.819172,0.809046,0.843733,0.843369
1200,0.636700,1.131512,0.820765,0.811212,0.841078,0.840804
1400,0.621300,1.099803,0.823287,0.814220,0.848779,0.848466


TrainOutput(global_step=1416, training_loss=0.878194627115282, metrics={'train_runtime': 160.1311, 'train_samples_per_second': 141.309, 'train_steps_per_second': 8.843, 'total_flos': 3039654752954112.0, 'train_loss': 0.878194627115282, 'epoch': 2.0})

## Text Generaion

In [24]:
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
distilgpt2 = AutoModelForCausalLM.from_pretrained("distilgpt2").to("cuda")

generator = pipeline(task="text-generation", model=distilgpt2, tokenizer=AutoTokenizer.from_pretrained("distilgpt2"), device="cuda:0")
generator("Welcome, students! I am DistilGPT2 and today I want to tell you that I can")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Welcome, students! I am DistilGPT2 and today I want to tell you that I can tell you that I am not my best friend, just as you have all of you on my facebook page. It would be nice if he didn'}]

Посмотрим внимательнее как работает gpt2. Давайте попробуем сгенерировать продолжение для нашего текста:

In [26]:
text = "I don't think that you" # текст, который мы хотим продолжить
encoded = tokenizer.encode(text, return_tensors='pt').cuda()
output = distilgpt2(encoded)
output.logits.shape

torch.Size([1, 6, 50257])

Видим, что нет CLS или SEP, зато есть EOS BOS (начало и конец предложения, соответственно). Попробуем чуть внимательней глянуть на генерацию текста

In [27]:
from transformers.models.gpt2 import GPT2Tokenizer

probabilities = torch.nn.Softmax(dim=0)(output.logits[0][-1]) # софтмакс берем только для интерпретируемости

tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2") # вручную возьмем не-фаст токенайзер, ибо его проще "вскрывать" и доставать его данные

token_probabilities = {t: p for t, p in zip(tokenizer.encoder, probabilities)}
sorted_probabilities = sorted(token_probabilities.items(), key=lambda x: -x[1]) # сортируем, получая список возможных слов, стоящих после you
print(sorted_probabilities[:5]) # смотрим 5 наиболее вероятных слов

[('Ġcan', tensor(0.1870, device='cuda:0', grad_fn=<UnbindBackward0>)), ("'re", tensor(0.1553, device='cuda:0', grad_fn=<UnbindBackward0>)), ('Ġshould', tensor(0.0689, device='cuda:0', grad_fn=<UnbindBackward0>)), ('Ġare', tensor(0.0634, device='cuda:0', grad_fn=<UnbindBackward0>)), ('Ġhave', tensor(0.0604, device='cuda:0', grad_fn=<UnbindBackward0>))]


Закорючка в виде G - пробел. Для примера скажем, что мы выбрали " are" в качестве продолжения. Генерируем дальше:

In [17]:
encoded = tokenizer.encode(" are", return_tensors='pt') # новая присобаченная последовательность - только ' are', весь предыдущий текст тащить не нужно
new_output = distilgpt2(encoded, past_key_values=output[1]) # засовываем то, что насчитали для исходной последовательности, в past_key_value

probabilities = torch.nn.Softmax(dim=0)(new_output.logits[0][-1]) # пересчитываем вероятности


token_probabilities = {t: p for t, p in zip(tokenizer.encoder, probabilities)}
sorted_probabilities = sorted(token_probabilities.items(), key=lambda x: -x[1])
print(sorted_probabilities[:5])

[('Ġa', tensor(0.1224, grad_fn=<UnbindBackward0>)), ('Ġgoing', tensor(0.0945, grad_fn=<UnbindBackward0>)), ('Ġthe', tensor(0.0620, grad_fn=<UnbindBackward0>)), ('Ġdoing', tensor(0.0273, grad_fn=<UnbindBackward0>)), ('Ġin', tensor(0.0193, grad_fn=<UnbindBackward0>))]


In [31]:
prompt = 'What is the best anime of all times?.\n'
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

generate_ids = distilgpt2.generate(inputs.input_ids.cuda(), max_length=100)

print(tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is the best anime of all times?.


What was your favorite film from this article?


Вот и GPT когда-то ничего не умел:)